<a href="https://colab.research.google.com/github/steve-keys/Assignment---1--Intro-to-Programming/blob/main/assignment_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Begin by loading the packages and importing the functions required for the subsequent work flow and algorithims
!pip install kora -q

'''load packages and import modules'''

from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

     |████████████████████████████████| 57 kB 2.5 MB/s 
     |████████████████████████████████| 56 kB 3.3 MB/s 


In [3]:
def get_ticker():
  '''
  This function will prompt the user for their company's stock market ticker symbol and build the url for the relevant web page
  '''
  company_ticker = input("What is your Company's Stock Market Ticker Symbol? ")
  url = "https://au.finance.yahoo.com/quote/{}/profile?p={}".format(company_ticker,company_ticker)
  return url

* Need to research how to validate input as 3 letters followed by a dot AX - i.e. ABC.AX 
* Not case sensitive

In [4]:
def get_name_title_pay(row):
  '''
  This function will obtain the name, title and pay (first 3 cells) for each Executive listed in the profile section of the web page
  '''
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title':title, 'pay': pay}

In [5]:
def get_exec_details(soup):
  '''
  This function will scour the web page soup for the table, body and rows, and then return the name, title and pay for each Executive in each of the rows (10 rows in total)
  '''
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  
  exec_details = []
  for row in rows:
    result = get_name_title_pay(row)
    exec_details.append(result)
  return exec_details

In [6]:
def get_salary(rec):
  '''
  This function will convert the Executives salaries into a full number (float format)
  '''
  pay = rec['pay']
  last_char = pay[-1]
  num_part = pay[0:-1]

  if last_char == 'M':
    salary = float(num_part) * 1000000
  elif last_char == 'k':
    salary = float(num_part) * 1000
  else:
    salary = 0

  return salary

In [7]:
def calculate_tax (income):
  '''
  This function will calculate the income tax payable on each persons salary
  '''
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000: 
    tax_payable = 0.19 * (income - 18200)
  elif 45000 < income <= 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
  elif 120000 < income <= 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
  elif income > 180000:     
    tax_payable = 51667 + 0.45 * (income - 180000)        
  
  return tax_payable

In [8]:
''' 
This set of functions will fetch the web page, parse the soup from that page, retrieve the data needed to calculate the income tax payable and finally for each Executive, save their name, title, pay and tax liability (adding a column for this tax), all into a CSV file that presents as a table
'''
 
url = get_ticker()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_exec_details(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

#create a pandas dataframe - as shown in an earlier notebook a df is easily saved as a CSV file

execs_df = pd.DataFrame.from_records(records)
execs_df

What is your Company's Stock Market Ticker Symbol? nab.ax


,name,title,pay,tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2.54M,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1.13M,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1.72M,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912.98k,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929.27k,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1.62M,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1.05M,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333.27k,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,N/A,0.0
9,Ms. Sally Mihell,Head of Investor Relations,N/A,0.0


In [ ]:
## Need to also save data to a SQL Data Base
## Need to add GUI buttons to provide user with a choice to save to CSV or SQL

## THINGS LEFT TO DO
1. Need to save data to a SQL Data Base
2. Need to add GUI buttons to provide user with a choice to save to CSV or SQL
3. Need to validate company stock ticker input 

In [11]:
df = execs_df

df.to_sql(name='execs_df', con=conn)
conn.close()

NameError: ignored